Nombre: Ulises Gallardo Rodríguez

Tarea #3 Segundo Parcial

In [181]:
import numpy as np
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

In [182]:
def get_texts_from_file(path_corpus, path_truth):
    tr_txt = []
    tr_y = []
    
    with open(path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
        for tuit in f_corpus:
            tr_txt += [tuit]
            
        for label in f_truth:
            tr_y += [label]
            
    return tr_txt, tr_y

In [183]:
tr_txt, tr_y = get_texts_from_file("./mex_train.txt", "./mex_train_labels.txt")

In [184]:
print(len(tr_txt), " - ", len(tr_y))

5544  -  5544


In [185]:
for i in range(len(tr_txt)):
    tr_txt[i]= tr_txt[i].lower()

## N-grams

In [279]:
def n_grams_counts(tr_txt, n):
    Count = {}
    for i in range(len(tr_txt)):
        tweet = tr_txt[i]
        start = ""
        end = " </s>"
        for i in range(max(1,n-1)):
            start += "<s> "
        tweet = start+tweet+end 
        words = tokenizer.tokenize(tweet)    
        for index, word in enumerate(words[n-1:]):
            context = word
            for j in range(1,n):
                context = words[(index + n -1) - j] + " " + context
            if Count.get(context) is not None:
                Count[context] += 1
            else:
                Count[context] = 1
    return Count

In [309]:
def get_probabilities(Count_n_prev_gram, Count_n_gram, n, tr_txt_size):
    is_unigram = np.isscalar(Count_n_prev_gram)
    Probablilities = {}

    start = ""
    for i in range(max(1,n-1)):
        start += "<s>"
        if i<n-2:
            start+=" "
    #print(start)

    if is_unigram:
        for word, count in Count_n_gram.items():
            Probablilities[word] = count/Count_n_prev_gram
    else:
        for word, count in Count_n_gram.items():
            s = word
            last = word.split(" ").pop()
            s = s[:-(len(last)+1)].rstrip()
            #print(word,"-->", s)
            try :
                if s == start:
                    count_prev = tr_txt_size
                else :
                    count_prev = Count_n_prev_gram[s]
                Probablilities[word] = count / count_prev
            except:
                print("error en:",s)
    return Probablilities

In [52]:
"""
    indices_palabras["<s>"] = len(indices_palabras)+1
    indices_palabras["</s>"] = len(indices_palabras)+1
    indices_palabras["<UNK>"] = len(indices_palabras)+1
    V_size+=3 
"""

'\n    indices_palabras["<s>"] = len(indices_palabras)+1\n    indices_palabras["</s>"] = len(indices_palabras)+1\n    indices_palabras["<UNK>"] = len(indices_palabras)+1\n    V_size+=3 \n'

## Corpus de Prueba

In [282]:
tr_prueba, tr_y_prueba = get_texts_from_file("./prueba.txt", "./prueba.txt")

In [283]:
corpus_palabras = []
for doc in tr_prueba:
    corpus_palabras += tokenizer.tokenize(doc)

In [284]:
corpus_palabras

['I',
 'am',
 'Sam',
 'Sam',
 'I',
 'am',
 'I',
 'do',
 'not',
 'like',
 'green',
 'eggs',
 'and',
 'ham']

In [285]:
indices_palabras = dict()
cnt = 0
V = set(corpus_palabras)
for word in V:
    indices_palabras[word]=cnt
    cnt += 1  

In [286]:
indices_palabras

{'do': 0,
 'not': 1,
 'green': 2,
 'I': 3,
 'am': 4,
 'ham': 5,
 'Sam': 6,
 'eggs': 7,
 'and': 8,
 'like': 9}

In [298]:
Count_unigram = n_grams_counts(tr_prueba, 1)
Count_bigram = n_grams_counts(tr_prueba, 2)
Count_trigram = n_grams_counts(tr_prueba, 3)

In [299]:
Count_unigram

{'<s>': 3,
 'I': 3,
 'am': 2,
 'Sam': 2,
 '</s>': 3,
 'do': 1,
 'not': 1,
 'like': 1,
 'green': 1,
 'eggs': 1,
 'and': 1,
 'ham': 1}

In [293]:
Count_bigram

{'<s> I': 2,
 'I am': 2,
 'am Sam': 1,
 'Sam </s>': 1,
 '<s> Sam': 1,
 'Sam I': 1,
 'am </s>': 1,
 'I do': 1,
 'do not': 1,
 'not like': 1,
 'like green': 1,
 'green eggs': 1,
 'eggs and': 1,
 'and ham': 1,
 'ham </s>': 1}

In [306]:
get_probabilities(Count_unigram, Count_bigram, 2, len(tr_prueba))

<s>


{'<s> I': 0.6666666666666666,
 'I am': 0.6666666666666666,
 'am Sam': 0.5,
 'Sam </s>': 0.5,
 '<s> Sam': 0.3333333333333333,
 'Sam I': 0.5,
 'am </s>': 0.5,
 'I do': 0.3333333333333333,
 'do not': 1.0,
 'not like': 1.0,
 'like green': 1.0,
 'green eggs': 1.0,
 'eggs and': 1.0,
 'and ham': 1.0,
 'ham </s>': 1.0}

In [310]:
get_probabilities(Count_bigram, Count_trigram,3, len(tr_prueba))

{'<s> <s> I': 0.6666666666666666,
 '<s> I am': 0.5,
 'I am Sam': 0.5,
 'am Sam </s>': 1.0,
 '<s> <s> Sam': 0.3333333333333333,
 '<s> Sam I': 1.0,
 'Sam I am': 1.0,
 'I am </s>': 0.5,
 '<s> I do': 0.5,
 'I do not': 1.0,
 'do not like': 1.0,
 'not like green': 1.0,
 'like green eggs': 1.0,
 'green eggs and': 1.0,
 'eggs and ham': 1.0,
 'and ham </s>': 1.0}

## Corpus Tweets

In [311]:
corpus_palabras = []
for doc in tr_txt:
    corpus_palabras += tokenizer.tokenize(doc)

In [312]:
len(set(corpus_palabras))

13581

In [313]:
indices_palabras = dict()
cont = 0
V = set(corpus_palabras)
for word in V:
    indices_palabras[word]=cont
    cont += 1    

In [314]:
Count_unigram = n_grams_counts(tr_txt, 1)
Count_bigram = n_grams_counts(tr_txt, 2)
Count_trigram = n_grams_counts(tr_txt, 3)

In [316]:
bigram = get_probabilities(Count_unigram, Count_bigram, 2, len(tr_txt))

error en: pasar 556 816
error en: ensalada . .
error en: aberrante .
error en: mundial ...
error en: ! . . .
error en: ) . .
error en: tampoco . . .
error en: tarugos . .
error en: rico 552 032


In [317]:
trigram = get_probabilities(Count_bigram, Count_trigram, 3 , len(tr_txt))

error en: a pasar 556 816
error en: una ensalada . .
error en: algo aberrante .
error en: del mundial ...
error en: oook ! . . .
error en: doy ) . .
error en: gobernadores tampoco . . .
error en: . tarugos . .
error en: cogernos rico 552 032


In [318]:
Count_trigram

{'<s> <s> lo': 48,
 '<s> lo peor': 4,
 'lo peor de': 2,
 'peor de todo': 1,
 'de todo es': 1,
 'todo es que': 1,
 'es que no': 8,
 'que no me': 19,
 'no me dan': 1,
 'me dan por': 1,
 'dan por un': 1,
 'por un tiempo': 2,
 'un tiempo y': 1,
 'tiempo y luego': 1,
 'y luego vuelven': 1,
 'luego vuelven estoy': 1,
 'vuelven estoy hasta': 1,
 'estoy hasta la': 31,
 'hasta la verga': 15,
 'la verga de': 20,
 'verga de estl': 1,
 'de estl </s>': 1,
 '<s> <s> a': 118,
 '<s> a la': 16,
 'a la vga': 1,
 'la vga no': 1,
 'vga no seas': 1,
 'no seas mamón': 2,
 'seas mamón 45': 1,
 'mamón 45 putos': 1,
 '45 putos minutos': 1,
 'putos minutos después': 1,
 'minutos después me': 1,
 'después me dices': 1,
 'me dices que': 3,
 'dices que apenas': 1,
 'que apenas sales': 1,
 'apenas sales no': 1,
 'sales no me': 1,
 'no me querías': 1,
 'me querías avisar': 1,
 'querías avisar en': 1,
 'avisar en 3': 1,
 'en 3 horas': 1,
 '3 horas ?': 1,
 'horas ? 😑': 1,
 '? 😑 </s>': 1,
 '<s> <s> considero': 1,
 '<s>

In [319]:
trigram

{'<s> <s> lo': 0.008658008658008658,
 '<s> lo peor': 0.08333333333333333,
 'lo peor de': 0.16666666666666666,
 'peor de todo': 0.3333333333333333,
 'de todo es': 0.09090909090909091,
 'todo es que': 0.3333333333333333,
 'es que no': 0.1,
 'que no me': 0.08597285067873303,
 'no me dan': 0.00847457627118644,
 'me dan por': 0.07692307692307693,
 'dan por un': 1.0,
 'por un tiempo': 0.08695652173913043,
 'un tiempo y': 0.3333333333333333,
 'tiempo y luego': 0.25,
 'y luego vuelven': 0.05,
 'luego vuelven estoy': 1.0,
 'vuelven estoy hasta': 1.0,
 'estoy hasta la': 0.96875,
 'hasta la verga': 0.18072289156626506,
 'la verga de': 0.0477326968973747,
 'verga de estl': 0.03571428571428571,
 'de estl </s>': 1.0,
 '<s> <s> a': 0.021284271284271284,
 '<s> a la': 0.13559322033898305,
 'a la vga': 0.0025575447570332483,
 'la vga no': 1.0,
 'vga no seas': 1.0,
 'no seas mamón': 0.1111111111111111,
 'seas mamón 45': 0.5,
 'mamón 45 putos': 1.0,
 '45 putos minutos': 1.0,
 'putos minutos después': 0.25